<a href="https://colab.research.google.com/github/gabrielkmbo/bird-classifier/blob/main/bird_classifier_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLab Onboarding Project, Fall 2023

Your task is to classify different species of birds! See the project document pinned in \#mlab-general for more details.

## Setup

In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# workshop folder, e.g. 'acmlab/project'
FOLDERNAME = 'acmlab/project'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
PATH = '/content/drive/My Drive/{}'.format(FOLDERNAME)
sys.path.append(PATH)

%cd $PATH

Mounted at /content/drive
/content/drive/My Drive/acmlab/project


In [ ]:
# Math/deep learning libraries
import numpy as np
import torch
from torch import nn

# Data processing
import pandas as pd

# Data transformations
import torchvision

# Loading images/bundles of images
from PIL import Image
import h5py

# Plotting
import matplotlib.pyplot as plt

# Progress bars
import tqdm

# Other
import time
from datetime import date

In [ ]:
# Set PyTorch to use the GPU
device = torch.device('cuda')

## Dataset

You'll need to create a way to load our training images into PyTorch.

Ordinarily, you would load images one at a time from a folder on your computer (or in our case, Google Drive). However, since Google Drive's file reading speed is extremely slow (on the order of 1 hour for the training dataset), this is inefficient for developing a model. Note that this *wouldn't* be the case if we were running locally, where file reading is much faster.

Since we still want you to get experience with creating an image dataset, we'll have you create it the traditional way (by loading images one at a time) on a **small dataset** that won't be used for actual training.

For the actual training dataset, we created an `h5py` bundle of the data, which allows you to read a single (larger) file to get a full list of images. Instead of reading the images individually, you'll be able to use the functions provided in `utils.py` to load lists of the images, labels, and classes.

### Loading images individually

Fill in the following class to load images one by one into a dataset. We provide a `csv_path` as an argument.

1. Load the CSV file using Pandas. The CSV is formatted as follows:
```
filepath,label,class
small_data/0 - GREAT XENOPS/0_1.jpg,0,GREAT XENOPS
...
small_data/2 - OSPREY/2_15.jpg,2,OSPREY
```

2. For each row of the CSV, read in the image at the `filepath`, and add the image and label to the `self.images` and `self.labels` list.

  We can load images with `Image.open(filename)`.

  We'll also want to keep track of the classes themselves, so `self.classes[0]` should equal the name of the bird with label 0 (`"GREAT XENOPS"`).

3. Fill in the rest of the functions based on the `self.images`, `self.labels`, and `self.classes` variables.

In [ ]:
class BirdDatasetSmall(torch.utils.data.Dataset):
    def __init__(self, csv_path):
        self.images = []
        self.labels = []
        self.classes = []

        ######## BEGIN YOUR CODE HERE ########
        data = pd.read_csv(csv_path)
        for index, row in data.iterrows():
            image = Image.open(row['filepath'])
            self.images.append(image)
            self.labels.append(row['label'])
            if row['class'] not in self.classes:
                self.classes.append(row['class'])

        ######### END YOUR CODE HERE #########

        assert len(self.images) == len(self.labels)

    def __len__(self):
        """Returns the number of examples in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return len(self.images)
        ######### END YOUR CODE HERE #########
        return None

    def num_classes(self):
        """Returns the number of classes in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return len(self.classes)
        ######### END YOUR CODE HERE #########
        return None

    def get_class(self, label):
        """Returns the name of the bird corresponding to the given label value"""
        ######## BEGIN YOUR CODE HERE ########
        return self.classes[label]
        ######### END YOUR CODE HERE #########
        return None

    def get_image(self, idx):
        """Returns the image of the idx'th example in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return self.images[idx]
        ######### END YOUR CODE HERE #########
        return None

    def get_label(self, idx):
        """Returns the label of the idx'th example in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return self.labels[idx]
        ######### END YOUR CODE HERE #########
        return None

    def __getitem__(self, idx):
        """Returns a tuple of the image and label of the idx'th example in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return self.images[idx], self.labels[idx]
        ######### END YOUR CODE HERE #########
        return None

    def display(self, idx):
        """Displays the image at a given index"""
        display(self.get_image(idx))

#### Testing BirdDatasetSmall

First, we'll run some basic tests to make sure the dataset is reading the right values. The first time you run this cell, it might take about a minute to load the data.

In [ ]:
def test_case(message, value, expected_value):
    print(message)
    if value == expected_value:
        print("  PASSED")
    else:
        print(f"  EXPECTED: {expected_value}")
        print(f"  GOT: {value}")

small_dataset = BirdDatasetSmall("small_data.csv")

test_case("Length of the dataset", len(small_dataset), 45)
test_case("Label of idx=17", small_dataset.get_label(17), 1)

Length of the dataset
  PASSED
Label of idx=17
  PASSED


Next, we'll display some images from the dataset. The output of running the cell should match the following image:

![](https://drive.google.com/uc?export=view&id=1mqvn-KXh8mWV6HcxulAnNqDhIPPWCq6f)

In [ ]:
# Display some images from all_data
figure = plt.figure(figsize=(15, 10))
num_rows = 1
num_cols = 3

ds_idx = [13, 30, 43]
for plot_idx, idx in enumerate(ds_idx):
    ax = plt.subplot(num_rows, num_cols, plot_idx + 1) # subplot indices begin at 1, not 0
    ax.title.set_text(small_dataset.get_class(small_dataset.get_label(idx)))
    plt.axis('off')
    plt.imshow(small_dataset.get_image(idx))

### Loading images from a bundle

As mentioned previously, for Colab performance reasons we'll be loading all the training data from a pre-created HDF5 bundle.

You'll be filling in the same code as the previous dataset, but instead of loading images one by one, you will use the `utils.load_data()` function, which returns a tuple of a list of images, a list of labels, and a list of classes.

The other difference is the inclusion of a `transform` parameter. This is a `torchvision` Transform that should be applied to each image in `__getitem__` before returning.

In [ ]:
from utils import load_data

class BirdDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.images = []
        self.labels = []
        self.classes = []
        self.transform = None

        ######## BEGIN YOUR CODE HERE ########
        self.images, self.labels, self.classes = load_data()
        ######### END YOUR CODE HERE #########

        assert len(self.images) == len(self.labels)

    def __len__(self):
        """Returns the number of examples in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return len(self.images)
        ######### END YOUR CODE HERE #########
        return None

    def num_classes(self):
        """Returns the number of classes in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return len(self.classes)
        ######### END YOUR CODE HERE #########
        return None

    def get_class(self, label):
        """Returns the name of the bird corresponding to the given label value"""
        ######## BEGIN YOUR CODE HERE ########
        return self.classes[label]
        ######### END YOUR CODE HERE #########
        return None

    def get_image(self, idx):
        """Returns the image of the idx'th example in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return self.images[idx]
        ######### END YOUR CODE HERE #########
        return None

    def get_label(self, idx):
        """Returns the label of the idx'th example in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return self.labels[idx]
        ######### END YOUR CODE HERE #########
        return None

    def __getitem__(self, idx):
        """Returns a tuple of the image and label of the idx'th example in the dataset"""
        ######## BEGIN YOUR CODE HERE ########
        return self.images[idx], self.labels[idx]
        ######### END YOUR CODE HERE #########
        return None

    def display(self, idx):
        """Displays the image at a given index"""
        display(self.get_image(idx))

## Creating and visualizing the dataset

First, let's create the `BirdDataset`. Be sure to pass in the transform as a parameter. We're using a `ToTensor()` transform to ensure that when PyTorch goes through the dataset, it reads the images as Tensors instead of images.

The first time you run this cell, it'll take about a minute to load the file.

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

all_data = BirdDataset(transform)
#print(all_data.images)
#print(all_data.labels)
#print(all_data.classes)


### Visualizing the data

An important part of deep learning is to understand what type of data we're working with. Here, we'll visualize some of the examples of our training set.

In [ ]:
# Display some images from all_data
figure = plt.figure(figsize=(15, 10))
num_rows = 8
num_cols = 8
for plot_idx in range(20):
    ax = plt.subplot(num_rows, num_cols, plot_idx + 1) # subplot indices begin at 1, not 0
    idx = (plot_idx * 373) % len(all_data)  # Gets a "random" index in the dataset

    # for this code, use `idx` as the dataset index
    cur_image = all_data.images[idx]
    cur_label = all_data.labels[idx]
    cur_class = all_data.get_class(cur_label) # was I supposed to change this

    ax.title.set_text(cur_class)
    plt.axis('off')
    plt.imshow(cur_image)

## Model structure

Here's where you'll define the structure of your neural network. Refer back to earlier workshop notebooks for reference.

In [ ]:
class BirdModel(nn.Module):
    def __init__(self, num_classes=20):
        super().__init__()
        # input = 224 x 224 x 3
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(20 * 53 * 53, 50)
        self.fc2 = nn.Linear(50, 20)

    def forward(self, x):
        self.to(device)
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = x.reshape(-1, 20 * 53 * 53)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

## Training setup

### Hyperparameters

Here's where we'll define our hyperparameters. We put them all in this cell for ease of readability; if you choose to add more hyperparameters, we suggest you add them here.

As you're tuning your model, change these values how you see fit!

In [ ]:
batch_size = 32
learning_rate = 0.01
num_epochs = 50

### Data Augmentation

If you choose to use data augmentation, add your augmentation transforms here.

In [ ]:
aug = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0),
])

### Data loaders

Here we'll create the data loaders for the train and validation set. To split `all_data` into the training and validation sets, read the PyTorch documentation for `random_split`: https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split

Be sure to pass in the `batch_size` hyperparameter to the DataLoaders, and be sure to shuffle the training data loader!

In [ ]:
train_len = 0.7
val_len = 0.3

train_set, val_set = torch.utils.data.random_split(all_data, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

## Training

It's time to train our model!

### Evaluation function

First, let's create a function that evaluates the accuracy of a model on a specified dataset. We provide some of the skeleton code, but you'll be coding this one on your own!

Be sure to call `.to(device=device)` on any data tensors to make sure PyTorch is using the GPU.

In [ ]:
# split is a variable you can set as the "name" of the dataset (either "train" or "val")
def evaluate(model, data_loader, name="val"):
    model.to(device)
    correct = 0  # number of correct predictions
    total = 0    # total number of examples in the data loader

    ######## BEGIN YOUR CODE HERE ########
    with torch.no_grad():
      for images, labels in data_loader:
        images.to(device)
        labels.to(device)
        images = torch.permute(images, (0, 3, 1, 2))
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    ######### END YOUR CODE HERE #########

    print(f'Accuracy of the network on the {total} {name} images: {100 * correct / total}%')

### Training loop

This is it: this is where your model learns! You'll also be implementing this function on your own. A couple tips:
* Be sure to apply augmentations to your data if you're using them
* Be sure to call `.to(device=device)` on your model and any data tensors to make sure PyTorch is using the GPU
* Call `evaluate` on both the train and validation data loaders after each epoch

In [ ]:
model = BirdModel().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

start_time = time.time()
for epoch in range(num_epochs):

    epoch_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
      images = images.to(device=device)
      labels = labels.to(device=device)
      optimizer.zero_grad()
      images = torch.permute(images, (0, 3, 1, 2))
      outputs = model(images.float()).squeeze()

      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item() / labels.shape[0]
      if batch_idx % 100 == 0:
        print(f"Epoch {epoch}, batch {batch_idx}: {loss}")

      ######## BEGIN YOUR CODE HERE ########
        evaluate(model, train_loader, "train")
        evaluate(model, val_loader, "val")
      ######### END YOUR CODE HERE #########

end_time = time.time()
print(f"Total training time: {end_time - start_time} sec")

Epoch 0, batch 0: 3.0018577575683594
Accuracy of the network on the 2180 train images: 5.045871559633028%
Accuracy of the network on the 933 val images: 4.608788853161844%
Epoch 1, batch 0: 2.9938085079193115
Accuracy of the network on the 2180 train images: 7.568807339449541%
Accuracy of the network on the 933 val images: 7.288317256162915%
Epoch 2, batch 0: 3.0084657669067383
Accuracy of the network on the 2180 train images: 7.568807339449541%
Accuracy of the network on the 933 val images: 7.288317256162915%
Epoch 3, batch 0: 2.9813661575317383
Accuracy of the network on the 2180 train images: 7.568807339449541%
Accuracy of the network on the 933 val images: 7.288317256162915%
Epoch 4, batch 0: 3.033038854598999
Accuracy of the network on the 2180 train images: 7.568807339449541%
Accuracy of the network on the 933 val images: 7.288317256162915%
Epoch 5, batch 0: 2.9956228733062744
Accuracy of the network on the 2180 train images: 7.568807339449541%
Accuracy of the network on the 933 

### Saving model weights

Save your weights! Your final model's weights will be one component of your submission. We recommend saving weights with a recognizable name, such as the date and a short description of the structure of your model (e.g. `"2023_10_25_3LayerCNN"`)

In [ ]:
model_name = "model"  ### CHANGE THIS NAME EACH TIME YOU SAVE WEIGHTS! ###

date_str = date.today().strftime("%Y_%m_%d")
model_weights_filename = f"{date_str}_{model_name}.pt"

torch.save(model.state_dict(), f"{model_weights_filename}")
print(f"Model weights saved as {model_weights_filename}")

Model weights saved as 2023_12_02_model.pt


### Refining your model

Now that you've trained a basic model, it's time to see if you can improve its accuracy. Feel free to change anything – the model structure, hyperparameters, augmentation – or try employing training strategies like regularization or early stopping. Get creative!

## Predict functions

These functions will let us run the test set through your model. It is **incredibly important** that you implement these functions!

`predict` should load the image at the specified path, run it through the model (variable provided as a parameter), and output the **predicted label** (the single numerical value between 0 and 19) – *not* the class name (e.g. the name of the bird).

In [ ]:
def load_model():
    model_path = 'acmlab/project/2023_12_02_model.pt'
    model_path = '/content/drive/My Drive/{}'.format(model_path)
    model = BirdModel()
    model.load_state_dict(torch.load(model_path))
    return model

In [ ]:
def predict(model, image_path):
    prediction = None
    ######## BEGIN YOUR CODE HERE ########
    model.eval()
    img = Image.open(image_path)
    with torch.no_grad():
        x = torchvision.transforms.functional.to_tensor(img)
        x.unsqueeze_(0)
        #print(x.shape)
        x = x.cuda()

        output = model(x)
        index = torch.argmax(output)
        prediction = index.item()
    ######### END YOUR CODE HERE #########
    return prediction

In [ ]:
model = load_model()
FILENAME = 'acmlab/project/small_data/1 - KIWI/1_1.jpg'
img_path = '/content/drive/My Drive/{}'.format(FILENAME)
print(predict(model, img_path))

16
